# Bond FRN Analytics and Cashflows

This notebook demonstrates how to access and use the **Bond** functionalities within the **LSEG Financial Analytics SDK**. 

The focus will be on how to price an FRN (Floating Rate Note).

**You will be able to:**
- Define an FRN with an ISIN code
- Define an FRN with an index RIC
- Evaluate and calculate the risk metrics
- Extract and analyze detailed bond cashflow schedules

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments` - for Bond instrument definitions and analytics
- This notebook uses external libraries *pandas, IPython*; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
# This notebook uses external libraries *pandas, IPython*; 
# please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code."

from lseg_analytics.pricing.instruments import bond as bd

import pandas as pd
import datetime as dt
from IPython.display import display

## Data Preparation

To define a Bond instrument you need to follow a structured 3-step process:

1. **Bond Definition** - Specify basic bond parameters (coupon rate, maturity date, face value, payment frequency)
2. **Bond Instrument Definition** - Create the instrument object for pricing
3. **Pricing Preferences** - Configure pricing parameters and market data settings

### Define FRN Bond with ISIN code

In [2]:
bond_definition_isin = bd.BondDefinition(
    instrument_code = "US064159YN00", # ISIN code
    issue_date = dt.datetime(2023, 1, 15), # Optional, if not defined the value comes from the instrument reference data
    end_date = dt.datetime(2028, 1, 15), # Optional, if not defined the value comes from the instrument reference data
    instrument_tag = "BOND_FRN_ISIN" # A user defined string to identify the instrument
)

### Define FRN Bond with an index

In [3]:
bond_definition_index = bd.BondDefinition(
    index_fixing_ric = "USDSOFR=", # RIC of the index
    interest_type = bd.InterestTypeEnum.FLOAT, # Floating rate
    interest_payment_frequency = bd.InterestPaymentFrequencyEnum.SEMI_ANNUAL, # Coupons paid every 6M
    notional_ccy = "USD", # USD denomination
    notional_amount = 1E6, # $1M face value
    interest_calculation_method = bd.InterestCalculationMethodEnum.DCB_30_360, # 30/360 interest day count basis
    payment_business_day_convention = bd.PaymentBusinessDayConventionEnum.MODIFIED_FOLLOWING,
    issue_date = dt.datetime(2023, 1, 15), # Optional, if not defined the value comes from the instrument reference data
    end_date = dt.datetime(2028, 1, 15), # Optional, if not defined the value comes from the instrument reference data
    instrument_tag = "BOND_FRN_INDEX", # A user defined string to identify the instrument
    payment_business_days = "USA" # US calendar
)

### Create the bond instruments from the definitions

In [4]:
bond_instruments = [
    bd.BondDefinitionInstrument(definition = bond_definition_isin),
    bd.BondDefinitionInstrument(definition = bond_definition_index)
]

# Configuring Pricing Parameters
pricing_params = bd.BondPricingParameters(
    valuation_date=dt.datetime(2023, 7, 18)
)

We select the below the following fields that are interesting for Bond pricing and cash-flows:

- **InstrumentTag**: Tag for identification
- **Notional currency**: Currency of the Bond
- **Market Value**: Present value of the Bond, it is also the **Dirty Price** of the bond
- **Report currency**: Currency of Bond valuation and metrics
- **Clean Price**: Bond price excluding accrued interest
- **Yield**: Yield to maturity of the Bond
- **Modified Duration**: Interest rate sensitivity measure
- **Convexity**: Second-order price sensitivity to yield changes
- **DV01**: Sensitivity of the market value to a 1bp parallel shift in the zero-coupon curve
- **Cash-flows**: detailed cash-flows schedule

In [5]:
basic_fields = "InstrumentTag, NotionalCcy, MarketValueInDealCcy, ReportCcy, MarketValueInReportCcy"
additional_fields = "cleanPricePercent, dirtyPricePercent, yieldPercent, ModifiedDuration, convexity, DV01Bp"
cashflow_fields = "CashFlows"
fields = basic_fields + "," + additional_fields + "," + cashflow_fields

## Request Execution

In [6]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instrument definitions for batch processing

try:
    response = bd.price(
        definitions = bond_instruments,
        pricing_preferences = pricing_params,
        fields = fields # If not set, all the possible fields will be displayed
    )

    # Display response structure information
    analytics_data = response['data']['analytics'][0]
    if analytics_data['error'] == {}:
        print("   Calculation successful!")
    else:
        print(f"   Pricing error: {analytics_data['error']}")
        
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise


   Pricing error: {'code': 'QPS-Pricer.3002', 'message': 'Market data error : No quotation available for Bond.'}


## Results Display

### Key Sections in the `response` JSON

 - **definitions**: Bond instrument setup (coupon rate, maturity date, notional amount, payment frequency), using the BondDefinition that we created.

 - **pricingPreferences**: Valuation date, price side (BID/ASK/MID/LAST), currency settings, settlement conventions.

 - **analytics**:
   - **tabularData**: Customizable list of analytics given in `fields` argument of price() function (`data`, `headers`, `statuses`)
   - **cashflows**: Detailed schedule of payments
   - **description**: Bond instrument summary and default fields used in the calculation
   - **nominalMeasures**: Main risk metrics calculated on Bond (`modifiedDuration`, `convexity`, `dv01Bp`)
   - **pricingAnalysis**: detailed price information on Bonds (`cleanPricePercent`, `dirtyPricePercent`, `yieldPercent`)
   - **valuation**: Detailed valuation of the Bond instru;ent with the below fields:
      - `MarketValueInDealCcy`: Present value of the Bond in deal currency
      - `MarketValueInReportCcy`: Present value of the Bond in report currency
   - **error**: Root cause of any issues, if encountered


We give below for the three deals the tabular data details, that include information on valuation and risk metrics. Cash-Flows are represented afterwards

### Valuation and Risk metrics

In [7]:
# Convert multi-bond analytics to DataFrame for better visualization
print("Multi-Bond Analytics Summary:")
columns = [item['name'] for item in response['data']['analytics'][0]['tabularData']['headers']]
response_data = [item['tabularData']['data'] for item in response['data']['analytics']]

response_df = pd.DataFrame(response_data, columns=columns)

# Set InstrumentTag as index for better readability
response_df.set_index('InstrumentTag', inplace=True)

# Round numerical values to 4 decimal places while keeping strings unchanged
response_df_rounded = response_df.copy()
response_df_rounded = response_df_rounded.drop('CashFlows', axis=1)
for col in response_df_rounded.columns:
    if response_df_rounded[col].dtype in ['float64', 'int64']:
        response_df_rounded[col] = response_df_rounded[col].round(4)

display(response_df_rounded.T)

Multi-Bond Analytics Summary:


InstrumentTag,BOND_FRN_ISIN,BOND_FRN_INDEX
NotionalCcy,USD,USD
MarketValueInDealCcy,None,None
ReportCcy,USD,USD
MarketValueInReportCcy,None,None
CleanPricePercent,None,None
DirtyPricePercent,None,None
YieldPercent,None,None
ModifiedDuration,None,None
Convexity,None,None
DV01Bp,None,None


##### Note : In order to visualize all the possible fields, one can not set it in the price method, and retrieve them

In [8]:
display(pd.DataFrame(response['data']['analytics'][0]['tabularData']['headers']))

,name,type
0,InstrumentTag,String
1,NotionalCcy,String
2,MarketValueInDealCcy,Float
3,ReportCcy,String
4,MarketValueInReportCcy,Float
5,CleanPricePercent,Float
6,DirtyPricePercent,Float
7,YieldPercent,Float
8,ModifiedDuration,Float
9,Convexity,Float


In [9]:
# Helper function to extract the cashflows from the response
def display_cashflows(bond_index):
    cashflows = response['data']['analytics'][bond_index]['cashflows']['cashFlows'][0]["payments"]
    dates = [cf['date'] for cf in cashflows]
    amounts = [cf['amount'] for cf in cashflows]
    events = [cf['event'] for cf in cashflows]
    events_type = [cf['type'] for cf in cashflows]
    rates = [cf['indexFixings'][0]["couponRatePercent"] if "indexFixings" in cf else 0 for cf in cashflows]

    cashflows_df = pd.DataFrame({'Dates': dates, "Cashflows": events, "Cashflow type": events_type, "Rate Percent": rates, "Amounts": amounts})
    display(cashflows_df)

### Cashflows: Bond with ISIN code

In [10]:
display_cashflows(0) # 0 is the index as the bond with ISIN was passed at the first position in the price method

,Dates,Cashflows,Cashflow type,Rate Percent,Amounts
0,2023-10-16,Interest,Float,5.64214,14261.722222
1,2024-01-16,Interest,Float,5.64214,14261.722222
2,2024-04-15,Interest,Float,5.64214,14261.722222
3,2024-07-15,Interest,Float,5.64214,14261.722222
4,2024-10-15,Interest,Float,5.64214,14261.722222
5,2025-01-15,Interest,Float,5.64357,14893.888889
6,2025-04-15,Interest,Float,5.64106,13789.111111
7,2025-07-15,Interest,Float,5.64214,14261.722222
8,2025-10-15,Interest,Float,5.64214,14261.722222
9,2026-01-15,Interest,Float,5.64357,14893.888889


### Cashflows: Bond with Index RIC

In [11]:
display_cashflows(1) # 1 is the index as the bond with Index RIC was passed at the seconde position in the price method

,Dates,Cashflows,Cashflow type,Rate Percent,Amounts
0,2024-01-16,Interest,Float,5.05,24939.022576
1,2024-07-15,Interest,Float,5.06,24987.804806
2,2025-01-15,Interest,Float,5.06,24987.804806
3,2025-07-15,Interest,Float,5.06,24987.804806
4,2026-01-15,Interest,Float,5.06,24987.804806
5,2026-07-15,Interest,Float,5.06,24987.804806
6,2027-01-15,Interest,Float,5.06,24987.804806
7,2027-07-15,Interest,Float,5.06,24987.804806
8,2028-01-18,Interest,Float,5.06,24987.804806
9,2028-01-18,Principal,Fixed,0.00,1000000.000000
